In [2]:
import requests
import pandas as pd
from itertools import product
from pandasql import sqldf # pip install pandasql

# ------------------------------
# 1. Hente data fra SSB
# ------------------------------
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    """Henter data fra SSB API og returnerer JSON-responsen."""
    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    response = requests.post(api_url, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Feil ved henting av data: {response.status_code}\n{response.text}")


# ------------------------------
# 2. Prosessere og strukturere data
# ------------------------------
def process_data(data):
    """Prosesserer JSON-data og returnerer en formatert Pandas DataFrame."""
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values
    
    # Pivot for å få én kolonne per måling
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()

    # Navngi kolonner tydelig
    df_pivot.columns = ["År og ukenr.", 
                        "Fersk laks - Kilospris", 
                        "Fersk laks - Vekt (tonn)", 
                        "Frosset laks - Kilospris", 
                        "Frosset laks - Vekt (tonn)"]

    return df_pivot


# ------------------------------
# 3. Beregne CO₂-utslipp
# ------------------------------
CO2_FACTORS = {
    "Fersk laks": 1.1,    # kg CO₂ per tonn-km (fly)
    "Frosset laks": 0.01  # kg CO₂ per tonn-km (skip)
}

DISTANCES = {
    "Fersk laks": 2000,
    "Frosset laks": 5000
}

def beregn_co2_utslipp(df):
    """Legger til CO₂-utslipp basert på vekt og transporttype."""
    # Smelt datasettet for lettere håndtering
    df_melted = pd.melt(
        df, 
        id_vars=["År og ukenr."], 
        value_vars=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"],
        var_name="Type", 
        value_name="Vekt (tonn)"
    )
    df_melted.dropna(subset=["Vekt (tonn)"], inplace=True)

    # Trekk ut transporttype
    df_melted["Transporttype"] = df_melted["Type"].apply(lambda x: "Fersk laks" if "Fersk" in x else "Frosset laks")

    # Beregn CO₂-utslipp med list comprehension
    df_melted["CO2-utslipp (kg)"] = [
        vekt * CO2_FACTORS[typ] * DISTANCES[typ]
        for vekt, typ in zip(df_melted["Vekt (tonn)"], df_melted["Transporttype"])
    ]
    
    return df_melted


# ------------------------------
# 4. Bruk av Pandas SQL for analyse
# ------------------------------
def total_utslipp_per_uke(df):
    """Oppsummerer CO₂-utslipp per uke med SQL."""
    query = """
        SELECT 
            `År og ukenr.`, 
            Transporttype, 
            SUM(`CO2-utslipp (kg)`) AS `Total CO2-utslipp (kg)`
        FROM df
        GROUP BY `År og ukenr.`, Transporttype
        ORDER BY `År og ukenr.`
    """
    return sqldf(query)


# ------------------------------
# 5. Kjør alt sammen
# ------------------------------
def main():
    try:
        # Hent og prosesser data
        data = fetch_data()
        df_pivot = process_data(data)

        # Rens: fjern rader med NaN i vekt, som er nullverdier
        df_clean = df_pivot.dropna(subset=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"], how='all')

        # Definerer CO₂ utslipp 
        df_with_co2 = beregn_co2_utslipp(df_clean) 

        # defienrer sammenfattet data
        df_summary = total_utslipp_per_uke(df_with_co2)

        return df_summary

    except Exception as e:
        print("Feil:", e)
        return None


df_summary = main()
print(df_summary)

#if __name__ == "__main__":
    #main()
    



     År og ukenr. Transporttype  Total CO2-utslipp (kg)
0         2000U01    Fersk laks               8201600.0
1         2000U01  Frosset laks                 19150.0
2         2000U02    Fersk laks               8918800.0
3         2000U02  Frosset laks                 10800.0
4         2000U03    Fersk laks               8894600.0
...           ...           ...                     ...
2631      2025U12  Frosset laks                 23650.0
2632      2025U13    Fersk laks              41421600.0
2633      2025U13  Frosset laks                 28000.0
2634      2025U14    Fersk laks              41916600.0
2635      2025U14  Frosset laks                 30600.0

[2636 rows x 3 columns]


In [3]:
import plotly.io as pio
import plotly.express as px

fig = px.line(
    df_summary,
    x="År og ukenr.",
    y="Total CO2-utslipp (kg)",
    color="Transporttype",
    markers=True,
    title="Interaktivt CO₂-utslipp per uke for fersk og frossen laks",
    hover_data={
        "År og ukenr.": True,
        "Total CO2-utslipp (kg)": ":.0f",
        "Transporttype": True
    }
)

fig.update_traces(line=dict(width=2))  # litt tykkere linjer

fig.update_layout(
    width=1300,
    height=600,
    xaxis_title="Ukenummer (år+uke)",
    yaxis_title="CO₂-utslipp i kg",
    legend_title="Transporttype",
    title_x=0.5,
    xaxis_tickangle=-45
)

pio.show(fig)


In [ ]:
import random 
import requests
import pandas as pd
from itertools import product

# URL til SSB API for tabell 03024
api_url = "https://data.ssb.no/api/v0/no/table/03024/"

# Definerer API-spørringen
query = {
    "query": [],
    "response": {
        "format": "json-stat2"
    }
}

# Hente data fra API
response = requests.post(api_url, json=query)

# Sjekke om forespørselen var vellykket
if response.status_code == 200:
    data = response.json()
    
    # Ekstrahere dimensjonsnavn og verdier
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    
    # Opprette en liste av kombinasjoner av dimensjoner
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    
    # Generere alle kombinasjoner av dimensjonsverdier
    all_combinations = list(product(*dimensions))
    
    # Lage en DataFrame med riktige dimensjoner
    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values  # Knytte verdiene til riktige rader
    
    # Pivotere data for å få ønsket struktur
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()
    
    # Gi kolonnene mer forståelige navn med riktig rekkefølge
    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)", "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]
    



#Konverter "Tid" til str og fjern "U"
    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int) # Første 4 tegn er årstall
    df_pivot["Uke"] = df_pivot["Tid"].str[5:].astype(int)  # Etter "U" er uke

#Fjern den opprinnelige "Tid"-kolonnen
    df_pivot = df_pivot.drop(columns=["Tid"])

#Flytt "ÅrUke" først i datasettet
    df_pivot = df_pivot[["Uke"] + [col for col in df_pivot.columns if col not in ["Uke"]]]
    df_pivot = df_pivot[["År"] + [col for col in df_pivot.columns if col not in ["År"]]]

    
#Hvis du vil kombinere "År" og "Uke" til en kolonne men det er ikke lurt når man skal plotte data   
#Lag en ny kolonne "ÅrUke" med format YYYYWW
    #df_pivot["År/Uke"] = df_pivot["År"].astype(str) + df_pivot["Uke"].astype(str).str.zfill(2)
    #df_pivot = df_pivot.drop(columns=["År", "Uke"])



#Skriver ut data
    print(df_pivot.to_string(index=True))

    print("Data hentet, sortert og formatert for visning.")
else:
    print(f"Feil ved henting av data: {response.status_code}")
    print(response.text)  # Skriv ut feilmeldingen fra API-et




        År  Uke  Fersk laks - Kilospris  Fersk laks - Vekt (tonn)  Frosset laks - Kilospris  Frosset laks - Vekt (tonn)
0     2000    1                   30.98                    3728.0                     32.54                       383.0
1     2000    2                   31.12                    4054.0                     33.63                       216.0
2     2000    3                   31.03                    4043.0                     36.06                       633.0
3     2000    4                   30.95                    3730.0                     34.27                       393.0
4     2000    5                   31.30                    3831.0                     33.91                       453.0
5     2000    6                   32.53                    4415.0                     35.30                       265.0
6     2000    7                   32.46                    4617.0                     34.49                       344.0
7     2000    8                   32.19 